In [1]:
import numpy as np
import os
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator, load_img, img_to_array
from tensorflow.keras.applications.inception_v3 import InceptionV3, preprocess_input

/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/__init__.py:98: UserWarning: unable to load libtensorflow_io_plugins.so: unable to open file: libtensorflow_io_plugins.so, from paths: ['/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/libtensorflow_io_plugins.so']
caused by: ['/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/libtensorflow_io_plugins.so: undefined symbol: _ZN3tsl6StatusC1EN10tensorflow5error4CodeESt17basic_string_viewIcSt11char_traitsIcEENS_14SourceLocationE']
  warnings.warn(f"unable to load libtensorflow_io_plugins.so: {e}")
/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/__init__.py:104: UserWarning: file system plugins are not loaded: unable to open file: libtensorflow_io.so, from paths: ['/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/libtensorflow_io.so']
caused by: ['/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/libtensorflow_io.so: undefined symbol: _ZTVN10tenso

In [2]:
# Set the path to your dataset directory
dataset_dir = '/kaggle/input/identifying-disease-in-tea-leafs/tea sickness dataset'

# Set the image dimensions and batch size
image_height, image_width = 299, 299
batch_size = 32

In [3]:
# Create the ImageDataGenerator for preprocessing and augmentation
data_generator = ImageDataGenerator(preprocessing_function=preprocess_input)

# Load the images from the dataset directory
dataset_generator = data_generator.flow_from_directory(
    dataset_dir,
    target_size=(image_height, image_width),
    batch_size=batch_size,
    class_mode='categorical',
    shuffle=False
)

Found 885 images belonging to 8 classes.


In [4]:
# Get the number of classes from the dataset generator
num_classes = dataset_generator.num_classes

# Load the InceptionV3 model pre-trained on ImageNet
base_model = InceptionV3(include_top=False, weights='imagenet', input_shape=(image_height, image_width, 3))

# Freeze the base model layers
for layer in base_model.layers:
    layer.trainable = False

87910968/87910968 [==============================] - 3s 0us/step


In [5]:
# Build the model architecture
model = tf.keras.models.Sequential([
    base_model,
    tf.keras.layers.GlobalAveragePooling2D(),
    tf.keras.layers.Dense(1024, activation='relu'),
    tf.keras.layers.Dropout(0.5),
    tf.keras.layers.Dense(num_classes, activation='softmax')
])

In [6]:
# Compile the model
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

In [12]:
model.fit(dataset_generator, epochs=10)

Epoch 1/10
28/28 [==============================] - 34s 1s/step - loss: 0.2821 - accuracy: 0.9040
Epoch 2/10
28/28 [==============================] - 36s 1s/step - loss: 0.3227 - accuracy: 0.8644
Epoch 3/10
28/28 [==============================] - 34s 1s/step - loss: 0.2603 - accuracy: 0.9028
Epoch 4/10
28/28 [==============================] - 36s 1s/step - loss: 0.2451 - accuracy: 0.9040
Epoch 5/10
28/28 [==============================] - 34s 1s/step - loss: 0.2497 - accuracy: 0.9141
Epoch 6/10
28/28 [==============================] - 34s 1s/step - loss: 0.2610 - accuracy: 0.9085
Epoch 7/10
28/28 [==============================] - 34s 1s/step - loss: 0.2989 - accuracy: 0.8814
Epoch 8/10
28/28 [==============================] - 34s 1s/step - loss: 0.2715 - accuracy: 0.9006
Epoch 9/10
28/28 [==============================] - 34s 1s/step - loss: 0.3010 - accuracy: 0.8881
Epoch 10/10
28/28 [==============================] - 34s 1s/step - loss: 0.2695 - accuracy: 0.9119


In [17]:
# Make predictions on the test set
predictions = model.predict(dataset_generator)

28/28 [==============================] - 33s 1s/step


In [19]:
predicted_labels = np.argmax(predictions, axis=1)

# Get the class labels from the dataset generator
class_labels = list(dataset_generator.class_indices.keys())

In [20]:
# Print the predicted labels for each image
for i, image_path in enumerate(dataset_generator.filepaths):
    image_name = os.path.basename(image_path)
    predicted_label = class_labels[predicted_labels[i]]
    print(f"Image: {image_name}, Predicted Label: {predicted_label}")

Image: IMG_20220503_143242.jpg, Predicted Label: Anthracnose
Image: IMG_20220503_143328.jpg, Predicted Label: Anthracnose
Image: IMG_20220503_143336.jpg, Predicted Label: bird eye spot
Image: IMG_20220503_143344.jpg, Predicted Label: Anthracnose
Image: IMG_20220503_143352.jpg, Predicted Label: bird eye spot
Image: IMG_20220503_143401.jpg, Predicted Label: bird eye spot
Image: IMG_20220503_143424.jpg, Predicted Label: Anthracnose
Image: IMG_20220503_143433.jpg, Predicted Label: Anthracnose
Image: IMG_20220503_143441.jpg, Predicted Label: Anthracnose
Image: IMG_20220503_143451.jpg, Predicted Label: Anthracnose
Image: IMG_20220503_143501.jpg, Predicted Label: Anthracnose
Image: IMG_20220503_143511.jpg, Predicted Label: Anthracnose
Image: IMG_20220503_143525.jpg, Predicted Label: Anthracnose
Image: IMG_20220503_143604.jpg, Predicted Label: Anthracnose
Image: IMG_20220503_143613.jpg, Predicted Label: bird eye spot
Image: IMG_20220503_143630.jpg, Predicted Label: bird eye spot
Image: IMG_202